# GPU accelerated training

Using twinLab you can accelerate the training time of some models using our cloud-based GPU training. In our testing, this has only produced reliable speed-ups for larger variational Gaussian Processes (`estimator_type="variational_gp"`), or for models with output decomposition (either `output_explained_variance` or `output_retained_dimensions` set in `TrainParams`).

Start, as usual, by importing the packages you need for this tutorial:

In [1]:
import twinlab as tl


          ====== TwinLab Client Initialisation ======
          Version     : 2.20.1.dev0
          User        : alexander@digilab.co.uk
          Server      : http://localhost:8000
          Environment : /Users/mead/digiLab/twinLab/tutorials/.env



We use the `tritium-desorption` example for this tutorial:

In [2]:
dataset_id = "tritium-desorption-small"
dataset = tl.Dataset(dataset_id)
df = tl.load_example_dataset(dataset_id)
dataset.upload(df)

Now we setup the emulator to be trained. With this dataset we try to predict the desorption rate (a function of temperature, so this is a functional output model) as a function of parameters of the material used in the reactor wall (parametrized via `E1`, `E2`, `E3`, `n1`, `n2`). 

In [3]:
emulator_id = "tritium-desorption"
inputs = ["E1", "E2", "E3", "n1", "n2"]
outputs = [f"y{i}" for i in range(624)]
emulator = tl.Emulator(emulator_id)
params = tl.TrainParams(output_retained_dimensions=5)

# Create a dataset on which to make predictions
df_test = df[inputs].sample(5, random_state=42).reset_index(drop=True)

The GPU is activated by setting `processor="gpu"` in the call to `emulator.train`. Here we see that it cuts down training time by a factor of $\sim 1/2$. This is not a life-changing improvement, but an improvement never-the-less.

In [4]:
# Loop over CPU and GPU to compare training times between the two
for processor in ["cpu", "gpu"]:
    print(f"Training model on {processor}")
    emulator.train(
        dataset,
        inputs,
        outputs,
        params=params,
        verbose=True,
        processor=processor,
    )
    print()
    print(f"Predicting from trained {processor} model...")
    emulator.predict(df_test, verbose=False)
    print("Predictions made successfully!")
    print()

Training model on cpu
Emulator 'tritium-desorption' has begun training.
0:00:00: Job status: processing
0:00:03: Job status: processing
0:00:08: Job status: processing
0:00:17: Job status: processing
0:00:31: Job status: processing
0:00:36: Job status: processing
0:00:41: Job status: processing
0:00:47: Job status: processing
0:00:53: Job status: processing
0:01:00: Job status: processing
0:01:08: Job status: processing
0:01:16: Job status: processing
0:01:25: Job status: processing
0:01:35: Job status: processing
0:01:46: Job status: processing
0:01:59: Job status: processing
0:02:12: Job status: success
Training of emulator tritium-desorption is complete!

Predicting from trained cpu model...
Predictions made successfully!

Training model on gpu
Emulator 'tritium-desorption' has begun training.
0:00:00: Job status: processing
0:00:01: Job status: processing
0:00:03: Job status: processing
0:00:04: Job status: processing
0:00:06: Job status: processing
0:00:08: Job status: processing
